In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
#from transformers_xl import XLNetLMHeadModel
from datasets import load_dataset
#from transformers.integrations import TensorBoardCallback
import re
import torch
from torch import optim
import emoji
from emoji import demojize
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') 

2024-02-17 06:43:30.745711: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 06:43:30.745850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 06:43:30.873545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df['target'].value_counts(normalize='true')

target
0    0.57034
1    0.42966
Name: proportion, dtype: float64

In [6]:
data = df[['text', 'target']]

In [7]:
def preprocess(texts):
    """
    Performs basic preprocessing on a tweet text:
    - Replace emojis with their meaning using demojize
    - Remove punctuation
    - Remove stop words like "a", "the", "in"
    """
    #texts = data['text']
    #labels = data['target']
    
    # Remove urls
    pattern = r"(https?://)?(www\.)?(\S+\.\S+)(/\S*)?"
    texts = [re.sub(pattern, "", text) for text in texts]
    
    # Replace emojis with meaning
    texts = [emoji.demojize(text) for text in texts] 
    #text = remove_urls_complete(text)
    
    # Remove punctuation
    texts = [re.sub(r"[^\w\s]", "", text) for text in texts]

    stop_words = set(stopwords.words('english'))
    #words = text.split()]
    #filtered_words = [word for word in words if word not in stop_words]
    #text = " ".join(filtered_words)
    
    texts = [
        " ".join([word for word in text.split() if word not in stop_words])
        for text in texts
    ]
    
    texts = [text.lower() for text in texts]
    
    #tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
    #tokenized_inputs = tokenizer(texts, truncation=True, padding="max_length")
    #processed_texts.append(processed_text)
    return texts#{"input_ids": tokenized_inputs["input_ids"], "attention_mask": tokenized_inputs["attention_mask"], "labels": labels}

# Example usage
#tweet = "This movie was  so much fun! #comedy #mustwatch 🍞🍖 ain't http://t.co/o9qknbfOFXI  "
#cleaned_tweet = preprocess_function(tweet)
#print(cleaned_tweet)  # Output: "movie much fun comedy mustwatch"

In [8]:
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
data['text'] = preprocess(df['text'])

/tmp/ipykernel_34/1654619930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = preprocess(df['text'])


In [10]:
data.head()

,text,target
0,our deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,all residents asked shelter place notified off...,1
3,13000 people receive wildfires evacuation orde...,1
4,just got sent photo ruby alaska smoke wildfire...,1


In [11]:
data['target'] = data['target'].apply(lambda x: 1 if x==0 else 0)

/tmp/ipykernel_34/596488809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['target'].apply(lambda x: 1 if x==0 else 0)


In [12]:
data.head()

,text,target
0,our deeds reason earthquake may allah forgive us,0
1,forest fire near la ronge sask canada,0
2,all residents asked shelter place notified off...,0
3,13000 people receive wildfires evacuation orde...,0
4,just got sent photo ruby alaska smoke wildfire...,0


In [18]:
examples = data.to_dict()
text = examples["text"][0] + str(examples["target"][0])
text

'our deeds reason earthquake may allah forgive us0'

In [19]:
prompt_template_text = """### text:
{text}

### label:
{label}"""

In [20]:
tex = examples['text'][0]
tar = examples['target'][0]

text_with_promte = prompt_template_text.format(text = tex, label = tar)

In [21]:
text_with_promte

'### text:\nour deeds reason earthquake may allah forgive us\n\n### label:\n0'

In [13]:
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"

In [25]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length": 2048
    },
    "datasets": {
        "use_hf": False,
        "train": {
            "data": data,
            "text_column": "text",
            "label_column": "target"
        }
    },
    "verbose": True
}

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
num_epochs = 3 
learning_rate = 2e-5 
max_length = 15

In [241]:
def tokenize_text(text):
    encoded_text = tokenizer(text, return_tensors="pt", truncation=True, padding = True)
    return encoded_text.to(device)

In [242]:
training_data = [tokenize_text(text) for text in data['text']]
#training_labels = torch.tensor(data["target"])

In [243]:
training_data[0]

{'input_ids': tensor([[   101,  17446,  37246,  10107,  27949,  63406,  11387,  10512,  10237,
          10142, 106088,  19626,    102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [117]:
print(training_data[:5])

[[101, 17446, 37246, 10107, 27949, 63406, 11387, 10512, 10237, 10142, 106088, 19626, 102], [101, 27736, 13559, 12883, 10109, 25470, 18180, 10148, 11478, 10944, 11153, 102], [101, 10435, 22058, 22151, 85947, 11192, 75349, 19907, 23579, 10192, 103730, 75545, 10822, 85947, 11192, 31303, 25973, 102], [101, 20968, 10929, 11426, 26286, 32290, 38990, 10107, 103730, 75545, 10822, 31303, 25923, 13918, 26302, 10280, 102], [101, 12820, 19556, 14541, 38171, 13483, 11530, 54183, 10371, 100332, 32290, 38990, 10107, 10322, 10107, 11393, 102]]


In [127]:
training_data1 = [torch.tensor(text, dtype=torch.int64) for text in training_data]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.config.num_labels = 2
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [19]:
train_texts = data['text']
train_labels = data['target'] 

In [20]:
from transformers import AdamW

In [21]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
#loss_function = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [201]:
training_data[0]

tensor([[   101,  17446,  37246,  10107,  27949,  63406,  11387,  10512,  10237,
          10142, 106088,  19626,    102]], device='cuda:0')

In [220]:
la = training_labels[0]

In [162]:
la.to(device)

tensor(0, device='cuda:0')

In [181]:
data.head()

,text,target
0,our deeds reason earthquake may allah forgive us,0
1,forest fire near la ronge sask canada,0
2,all residents asked shelter place notified off...,0
3,13000 people receive wildfires evacuation orde...,0
4,just got sent photo ruby alaska smoke wildfire...,0


In [233]:
encoded_text = tokenizer(data['text'][0], return_tensors="pt", padding=True, truncation=True)
inputs = encoded_text.to(device)

In [250]:
le = data['target'][0]

In [251]:
le

0

In [236]:
inputs

{'input_ids': tensor([[   101,  17446,  37246,  10107,  27949,  63406,  11387,  10512,  10237,
          10142, 106088,  19626,    102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [247]:
training_data[0]

{'input_ids': tensor([[   101,  17446,  37246,  10107,  27949,  63406,  11387,  10512,  10237,
          10142, 106088,  19626,    102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [257]:
output = model(**training_data[0], labels = torch.tensor([le]).to(device))

In [ ]:
output = model(**inputs, labels = torch.tensor(le).to(device))

In [230]:
training_data[0]

tensor([[   101,  17446,  37246,  10107,  27949,  63406,  11387,  10512,  10237,
          10142, 106088,  19626,    102]], device='cuda:0')

In [258]:
output.logits

tensor([[ 0.0851, -1.0249,  1.1287]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [207]:
torch.tensor([le])

tensor([0])

In [212]:
la

tensor([0, 0, 0,  ..., 0, 0, 0])

In [260]:
criterion(output.logits,torch.tensor([le]).to(device))

tensor(1.4276, device='cuda:0', grad_fn=<NllLossBackward0>)

In [262]:
labels = train_labels

In [22]:
for epoch in range(5):
    outputs = []
    model.train()
    for text, label in zip(training_data, labels):
        batch_text = text.to(device)
       # batch_labels = labels.to(device)
        
        output = model(**text)#, labels = torch.tensor([label]).to(device))
        
        loss = criterion(output.logits,torch.tensor([label]).to(device) )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        outputs.append(torch.argmax(output.logits).item())
    accuracy = calcuate_accu(outputs, labels)
    print(f'epoch: {epoch} has an Accuracy of {accuracy}')

NameError: name 'training_data' is not defined

In [ ]:
output

In [336]:
torch.save(model.state_dict(), "sentiment_model.pt")

In [338]:
model.load_state_dict(torch.load("sentiment_model.pt"))

<All keys matched successfully>

In [275]:
data['text'][0]

'our deeds reason earthquake may allah forgive us'

In [290]:
def predict_sentiment(text):
    encoded_text = tokenize_text(text)
    #input_ids = torch.tensor([encoded_text]).to(device)
    with torch.no_grad():
        outputs = model(**encoded_text)
        pred = torch.argmax(outputs.logits)
    # Convert prediction to sentiment label (e.g., "positive", "negative")
    return pred


In [339]:
predicted_sentiments = []
for text in data["text"]:
    predicted_sentiment = predict_sentiment(text).item()  # Get the sentiment value
    predicted_sentiments.append(predicted_sentiment)

data["predict"] = predicted_sentiments

/tmp/ipykernel_34/993029256.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["predict"] = predicted_sentiments


In [323]:
count

340

In [320]:
data

,text,target,predict
0,our deeds reason earthquake may allah forgive us,0,0
1,forest fire near la ronge sask canada,0,0
2,all residents asked shelter place notified off...,0,0
3,13000 people receive wildfires evacuation orde...,0,0
4,just got sent photo ruby alaska smoke wildfire...,0,0
...,...,...,...
7608,two giant cranes holding bridge collapse nearb...,0,0
7609,aria_ahrary thetawniest the control wild fires...,0,0
7610,0104 utc5km s volcano hawaii,0,0
7611,police investigating ebike collided car little...,0,0


In [340]:
from sklearn.metrics import confusion_matrix

# Assuming `data['target']` contains true labels and `predicted_sentiments` contains predictions
confusion_matrix = confusion_matrix(data['target'], predicted_sentiments)
print(confusion_matrix)

[[3141  130]
 [  82 4260]]


In [341]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()/ len(data)
    return n_correct

In [342]:
calcuate_accu(data['target'], data['predict'])

0.9721528963614869

In [325]:
count = 0
for i,y in zip(data['target'], data['predict']):
    if i!= y:
        count+=1


In [319]:
count

340

In [308]:
count

7613

In [312]:
df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [281]:
te = tokenize_text(text)

In [285]:
torch.argmax(model(**te).logits)

tensor(0, device='cuda:0')

In [343]:
df_sub = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [344]:
df_sub.head(['text']

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [350]:
data_text = preprocess(df_sub['text'])

In [354]:
data_test = pd.DataFrame({'id': df_sub['id'], 'text': preprocess(df_sub['text'])})

In [355]:
data_test.head()

,id,text
0,0,just happened terrible car crash
1,2,heard earthquake different cities stay safe ev...
2,3,forest fire spot pond geese fleeing across str...
3,9,apocalypse lighting spokane wildfires
4,11,typhoon soudelor kills 28 china taiwan


In [359]:
predicted_sentiments = []
for text in data_test["text"]:
    predicted_sentiment = predict_sentiment(text).item()  # Get the sentiment value
    predicted_sentiments.append(predicted_sentiment)

data_test["target"] = predicted_sentiments

In [361]:
data_test.head()

,id,text,target
0,0,just happened terrible car crash,1
1,2,heard earthquake different cities stay safe ev...,0
2,3,forest fire spot pond geese fleeing across str...,0
3,9,apocalypse lighting spokane wildfires,0
4,11,typhoon soudelor kills 28 china taiwan,0


In [363]:
sub = data_test.drop(columns = ['text'])

In [364]:
sub.head()

,id,target
0,0,1
1,2,0
2,3,0
3,9,0
4,11,0


In [378]:
sub['target'] = sub['target'].apply(lambda x: 1 if x==0 else 0)

In [379]:
sub.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [380]:
sub.to_csv('submission.csv', index = False)

In [381]:
s = pd.read_csv('/kaggle/working/submission.csv')

In [382]:
s.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [356]:
df_sub = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [357]:
df_sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [61]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer(
          text,
          return_tensors="pt",
          truncation=True,
          #max_length=max_input_tokens
  )
  inputs = input_ids.to(device)

  # Generate
  #device = model.device
  output = model( **inputs, )

  # Decode
  #generated_label = tokenizer.batch_decode(generated_label, skip_special_tokens=True)

  # Strip the prompt
  #generated_label_answer = generated_text_with_prompt[0][len(text):]

  return output

In [54]:
training_config['datasets']['train']['data']['text'][0]

'our deeds reason earthquake may allah forgive us'

In [66]:
test_text = training_config['datasets']['train']['data']['text'][0]
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {training_config['datasets']['train']['data']['text'][1]}")
print("Model's answer: ")
print(inference(test_text, model, tokenizer))

Question input (test): our deeds reason earthquake may allah forgive us
Correct answer from Lamini docs: forest fire near la ronge sask canada
Model's answer: 
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0965, -0.9158,  0.8817]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [67]:
max_steps = 3

In [68]:
trained_model_name = f"nlp{max_steps}_steps"
output_dir = trained_model_name

In [70]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [74]:
model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [79]:
trainer = Trainer(
    model=model,
    #model_flops=model_flops,
    #total_steps=max_steps,
    args=training_args,
    train_dataset=training_config,
    #eval_dataset=test_dataset,
)

In [80]:
training_output = trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


KeyError: 1